## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/RegularSession/[0830]TextMining_WordEmbedding/TextMining_WordEmbedding

/content/drive/MyDrive/RegularSession/[0830]TextMining_WordEmbedding/TextMining_WordEmbedding


In [3]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,099 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [1,076 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,540 kB]
Ign:13 https://developer.download.nvidia.com/comput

In [4]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [6]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [7]:
corpus[:50]

['대학과 산업계, 공공연구기관에서 일하는 기술경영 전문가 10명 중 7명은 박근혜 정부의 과학기술 정책이 실패했다고 평가하는 것으로 나타났다',
 '10명 중 8명은 과학기술 전담 부처의 필요성에 공감했다',
 '기술경영경제학회는 3일 이 같은 내용을 담은 ‘차기 정부 과학기술혁신 정책에 대한 설문 조사 결과’를 발표했다',
 '지난 10월17~23일 이뤄진 이 설문 조사에는 대학과 산업계, 공공연구기관에 종사하는 학회 소속 회원 160명이 참여했다',
 '응답자 중 69.4%가 정부 교체 때마다 바뀐 과학기술 정책이 모두 실패했다고 답했다',
 '과학기술처에서 과학기술부, 교육과학기술부를 거쳐 미래창조과학부로 전담 부처 이름을 바꾸고 정책 기조가 바뀌면서 일관성 있는 과학기술 정책을 추진하지 못했다는 것이다',
 '응답자의 66.9%는 창조경제를 내세운 현 정부의 과학기술 정책이 실패했다고 평가했다',
 '성공한 과학기술 정책을 편 정부를 꼽아달라는 질문에 응답자의 65%(104명)가 노무현 정부를, 22%(35명)는 김대중 정부를 선택했다',
 '과학기술 정책에서 가장 실패한 정부는 응답자 절반 이상인 56%(104명)가 박근혜 정부를, 36%(58명)가 이명박 정부를 꼽았다',
 '응답자의 78.8%는 차기 정부의 과학기술 전담 부처 설립 필요성에 공감했다',
 '우리나라 기초연구의 문제가 국가혁신시스템에서 기초연구 전략이 부족하다는 지적이 나왔다',
 '과학기술정책연구원(STEPI)이 3일 한국과학기술회관에서 열린 `기초연구 경쟁력, 질적 혁명에서부터`라는 주제 정책포럼에서 이민형 STEPI 선임연구위원은 “정부가 풀뿌리 지원만 강조하는데 `왜 기초연구를 하는가`에 대한 근본적 질문이 없다”고 비판했다',
 '기초과학이 국가 성장동력 핵심으로 강조되면서 세계 각 국은 기초연구 투자를 확대하고 있다',
 '하지만 우리나라 기초연구 경쟁력은 영향을 미치는 성과가 부족해 과학적 진보, 인류복지 기여 등 국내 경제사회적 문제해결 등 목적을 명시할 필요가 있다

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

- 품사를 구하는 방법으로 진행하고자 하였습니다!! 우주와 관련된 내용인데 주로 명사들이 중요한 역할을 할 것이라는 생각이 들었고, 추가로 형용사와 동사까지 추출해내면 의미는 보존이 될 것이라는 생각이 들었습니다.

In [8]:
okt = Okt()

In [9]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence):
  sen = okt.pos(sentence, stem  = True, norm = True)
  return [word for word, pos in sen if (pos=='Noun')|(pos=='Verb')|(pos=='Adjective')|(pos=='Adverb')]

  

In [10]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [11]:
corpus_tokenized[:5]

[['대학',
  '산업',
  '연구기관',
  '일',
  '하다',
  '기술',
  '경영',
  '전문가',
  '명',
  '중',
  '명',
  '박근혜',
  '정부',
  '과학기술',
  '정책',
  '실패하다',
  '평가',
  '하다',
  '것',
  '나타나다'],
 ['명', '중', '명', '과학기술', '전담', '부처', '필요성', '공감', '하다'],
 ['기술',
  '학회',
  '이',
  '같다',
  '내용',
  '담다',
  '차기',
  '정부',
  '과학기술',
  '혁신',
  '정책',
  '대한',
  '설문',
  '조사',
  '결과',
  '를',
  '발표',
  '하다'],
 ['지난',
  '이뤄지다',
  '이',
  '설문',
  '조사',
  '대학',
  '산업',
  '연구기관',
  '종사',
  '하다',
  '학회',
  '소속',
  '회원',
  '명',
  '참여',
  '하다'],
 ['응답',
  '중',
  '가다',
  '정부',
  '교체',
  '때',
  '바뀌다',
  '과학기술',
  '정책',
  '모두',
  '실패하다',
  '답',
  '하다']]

# 2번

이제 word2vec 모델을 학습시켜 봅시다!
https://wikidocs.net/50739 

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

size : embedding vector의 차원 - 사전학습된 w2v을 쓸 때 200을 쓸 것이니 지금도 200으로 하고자 합니다.  
min_count = 1 (5로 하면 너무 많은 단어가 사라질 것 같아서 0으로 설정하고자 합니다!)  
window = 5 (default, 위의 토큰화된 것들을 보았을 때도 5개 정도면 의미관계를 찾기 충분해보입니다!)   
sg = 1, Skip Gram (default, skip gram이 더 좋다고 하셨으니까요..ㅎㅎ)   
hs = 1 (default,Hierarchical Softmax까지 사용하면 더 좋을테니까요...ㅎㅎ)   
negative : negative sampling 의 noise words 개수 (default = 5)   
alpha : 초기 learning rate (default = 0.025)   
min_alpha : 최소 learning_rate   
seed : random seed   

In [12]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size=200, min_count=0)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 3558


In [13]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 15)

(210396, 237690)

In [14]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('정책')
# 으음.....

[('등', 0.99991375207901),
 ('이', 0.9999111890792847),
 ('받다', 0.999908447265625),
 ('늘다', 0.9999063611030579),
 ('때', 0.9999049305915833),
 ('한국', 0.9999045133590698),
 ('분야', 0.9999043345451355),
 ('대학', 0.9999039173126221),
 ('정부', 0.9999037384986877),
 ('오다', 0.9999030828475952)]

In [16]:
model_no_pretrained.wv.most_similar('기술')
# 으음..... 잘 찾지는 못 하는 느낌이긴 합니다...

[('등', 0.9999741315841675),
 ('되다', 0.9999721050262451),
 ('이', 0.999971866607666),
 ('오다', 0.9999701976776123),
 ('교수', 0.9999699592590332),
 ('의', 0.9999686479568481),
 ('며', 0.9999685287475586),
 ('하다', 0.9999682903289795),
 ('만들다', 0.9999682903289795),
 ('로봇', 0.9999672174453735)]

In [17]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size=200, min_count=0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 3558


In [18]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 31219


In [19]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 15)

(234758, 237690)

In [20]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('정책')

[('시책', 0.7225658893585205),
 ('방침', 0.6422121524810791),
 ('개혁', 0.6123769879341125),
 ('방안', 0.6097024083137512),
 ('개혁안', 0.5898494720458984),
 ('전략', 0.5749492645263672),
 ('공약', 0.5591204166412354),
 ('이념', 0.5448523759841919),
 ('현안', 0.5357989072799683),
 ('법안', 0.5346407294273376)]

In [21]:
model_pretrained.wv.most_similar('기술')

[('기계학습', 0.7566412091255188),
 ('강력하다', 0.7550637125968933),
 ('나노입자', 0.7340494394302368),
 ('가능하다', 0.7303067445755005),
 ('도르다', 0.7302178144454956),
 ('지르다', 0.7292910218238831),
 ('즐리', 0.7289735078811646),
 ('의료기기', 0.7276889085769653),
 ('구글', 0.7272005081176758),
 ('나르다', 0.7243545055389404)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

사전학습 없이 모델을 학습할 때에는 원래 말뭉치에서 가지고 있는 단어로만 학습을 하기에 유사한 단어를 찾을 때 평소에도 많이 쓰이는 일반적인 단어가 나온 것 같고,  
사전학습이 된 모델을 사용했을 때에는 원래 말뭉치뿐만 아니라 사전학습된 다른 단어들도 포함하고 있어 유사한 단어들을 찾을 때 단어의 특징들이 잘 반영될 수 있던 것 같습니다.

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]  
큰 차이는 느끼지 못하겠지만, t-SNE가 PCA보다 조금 더 중앙으로 몰려있는 듯한 느낌입니다.  
PCA에서는 뭔가 위쪽에는 국가에서 하는 위원회 등등의 단어들이 가있는 것 같고 오른쪽으로는 기술이나 실험과 관련된 단어들이 몰려있고 왼쪽 아래는 장소 등이 몰려있는 것을 볼 수 있습니다!

In [26]:
import plotly
import plotly.graph_objs as go
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [22]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

3558

In [24]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i])

In [25]:
pca = PCA(random_state = 0).fit_transform(vector)[:,:2]
print(pca.shape)
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)[:, :2]
print(tsne.shape)

(3558, 2)
(3558, 2)


In [27]:
X, t = make_swiss_roll(n_samples = 3558, noise = 0)

In [28]:
color = (t - t.min()) / (t.max() - t.min())

In [35]:
# 시각화해보세요!
# PCA

data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = unique_text,
    mode='markers+text', 
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

In [36]:
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = unique_text, 
    mode='markers+text', 
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [32]:
perplexity_list = [5, 20, 35, 50]

for for_perplexity in perplexity_list:
    tsne = TSNE(n_components = 2, random_state = 0, perplexity = for_perplexity, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

    # 시각화
    data = go.Scatter(
        x=tsne[:,0],
        y=tsne[:,1],
        text = ['point #{}'.format(i) for i in range(len(word))],
        mode='markers',
        marker=dict(
            size=5,
            color=color,
            colorscale='Jet',
            line=dict(
                width=0.0
            ),
            opacity=0.8
        )
    )

    layout = go.Layout(
        title={
            'text': 'perplexity : ' + str(for_perplexity) + ', 반복 1000회'},
        autosize=False,
        width=1000,
        height=1000,
        margin=go.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
    )
    fig = go.Figure(data=[data], layout=layout)
    fig.show()



/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [33]:
perplexity_list = [55, 70, 85, 100]

for for_perplexity in perplexity_list:
    tsne = TSNE(n_components = 2, random_state = 0, perplexity = for_perplexity, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

    # 시각화
    data = go.Scatter(
        x=tsne[:,0],
        y=tsne[:,1],
        text = ['point #{}'.format(i) for i in range(len(word))],
        mode='markers',
        marker=dict(
            size=5,
            color=color,
            colorscale='Jet',
            line=dict(
                width=0.0
            ),
            opacity=0.8
        )
    )

    layout = go.Layout(
        title={
            'text': 'perplexity : ' + str(for_perplexity) + ', 반복 1000회'},
        autosize=False,
        width=1000,
        height=1000,
        margin=go.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
    )
    fig = go.Figure(data=[data], layout=layout)
    fig.show()



/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!